# H2OAutoML Plugin

Since H2O-3 `3.28.0.1`, users have the possibility to customize the `H2OAutoML` model selection engine by writing their own training steps as a Java plugin.

This tutorial was updated for H2O-3 `3.46.0.1`.

## How to write a simple plugin

To create such plugin, user simply needs to create a small project containing at least:
- an implementation of the `ai.h2o.automl.ModelingStepsProvider` interface.
- a file `META-INF/services/ai.h2o.automl.ModelingStepsProvider` with a entry for each of those implementations that need to be exposed to the service provider of the main `H2O-3` jar.

This folder contains such a plugin example:
```text
.
├── Makefile
├── java_plugin.ipynb
└── src
    ├── META-INF
    │   └── services
    │       └── ai.h2o.automl.ModelingStepsProvider
    └── my
        └── automl
            ├── MyDRFStepsProvider.java
            └── MyGLMStepsProvider.java
```

with `src/META-INF/services/ai.h2o.automl.ModelingStepsProvider`:
```text
my.automl.MyDRFStepsProvider
my.automl.MyGLMStepsProvider
```

and for example `MyDRFStepsProvider.java`:
```java
package my.automl;

import ai.h2o.automl.*;
import hex.grid.Grid;
import hex.tree.drf.DRFModel;
import hex.tree.drf.DRFModel.DRFParameters;
import water.Job;

import java.util.HashMap;
import java.util.Map;
import java.util.stream.IntStream;

import static ai.h2o.automl.ModelingStep.ModelStep.DEFAULT_MODEL_TRAINING_WEIGHT;


public class MyDRFStepsProvider implements ModelingStepsProvider<MyDRFStepsProvider.DRFSteps> {

  public static class DRFSteps extends ModelingSteps {

    static final String NAME = Algo.DRF.name();
    static abstract class DRFGridStep extends ModelingStep.GridStep<DRFModel> {

      DRFGridStep(String id, AutoML autoML) {
        super(NAME, Algo.DRF, id, autoML);
      }

      public DRFParameters prepareModelParameters() {
        DRFParameters drfParameters = new DRFParameters();
        drfParameters._sample_rate = 0.8;
        drfParameters._col_sample_rate_per_tree = 0.8;
        drfParameters._col_sample_rate_change_per_level = 0.9;
        return drfParameters;
      }
    }

    private ModelingStep[] grids = new ModelingStep[]{
            new DRFGridStep("grid_1", aml()) {
              @Override
              public Map<String, Object[]> prepareSearchParameters() {
                Map<String, Object[]> searchParams = new HashMap<>();
                searchParams.put("_ntrees", IntStream.rangeClosed(5, 1000).filter(i -> i % 50 == 0).boxed().toArray());
                searchParams.put("_nbins", IntStream.of(5, 10, 15, 20, 30).boxed().toArray());
                searchParams.put("_max_depth", IntStream.rangeClosed(3, 20).boxed().toArray());
                searchParams.put("_min_rows", IntStream.of(3, 5, 10, 20, 50, 80, 100).boxed().toArray());
                return searchParams;
              }
            },
    };

    public DRFSteps(AutoML autoML) {
      super(autoML);
    }

    @Override
    protected ModelingStep[] getGrids() {
      return grids;
    }

    @Override
    public String getProvider() {
      return NAME;
    }
  }

  @Override
  public String getName() {
    return "MyDRF";
  }

  @Override
  public DRFSteps newInstance(AutoML aml) {
    return new DRFSteps(aml);
  }
}


```

As shown above, writing a `ModelingStepsProvider` simply requires to implement 2 methods:
- `String getName()` returning the name of this provider, which should be unique among all the registered providers: default algo names like "GLM", "XGBoost", "GBM", "DRF" are already used by `H2O-3` and must be avoided.
- `T newInstance(AutoML aml)` returning an instance of `ai.h2o.automl.ModelingSteps`: this is the class defining the logic for the default models and/or the grids that the user wants to add to `H2O AutoML`.


## How to add the plugin to H2O-3

H2O AutoML plugins are simply discovered using [ServiceLoader](https://docs.oracle.com/javase/8/docs/api/java/util/ServiceLoader.html), so the only requirement is to make this plugin available on the classpath.

The simplest way is to create a jar, and add it to the classpath.
For example, from this directory, running
```bash
make dist
```
will create a jar for our plugin in the `./dist` subfolder.

This jar can then be added to the classpath when starting `H2O-3`:
```bash
java -cp /path/to/h2o.jar:/path/to/automl/plugin.jar water.H2OApp
```
or directly from the clients:
- Python:
```python
import h2o
h2o.init(extra_classpath=["/path/to/automl/plugin.jar"])
```
- R:
```R
library("h2o")
h2o.init(extra_classpath=c("/path/to/automl/plugin.jar"))
```

In [1]:
# run this cell if you don't have h2o installed in your Python environment
!pip install h2o

  Using cached h2o-3.46.0.1-py2.py3-none-any.whl.metadata (2.1 kB)
  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
Using cached h2o-3.46.0.1-py2.py3-none-any.whl (265.6 MB)
Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)


In [2]:
# let's build our plugin jar
!make dist

rm -Rf ./build ./dist
sources = ./src/my/automl/MyGLMStepsProvider.java ./src/my/automl/MyDRFStepsProvider.java
mkdir -p build
javac ./src/my/automl/MyGLMStepsProvider.java ./src/my/automl/MyDRFStepsProvider.java -cp "/Users/tomasfryda/sources/h2o-tutorials/tutorials/automl/venv/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar" -d ./build
cp -R ./src/META-INF ./build
mkdir -p dist
jar cvf ./dist/h2oautoml_plugin.jar -C ./build .
added manifest
ignoring entry META-INF/
adding: META-INF/services/(in = 0) (out= 0)(stored 0%)
adding: META-INF/services/ai.h2o.automl.ModelingStepsProvider(in = 59) (out= 40)(deflated 32%)
adding: my/(in = 0) (out= 0)(stored 0%)
adding: my/automl/(in = 0) (out= 0)(stored 0%)
adding: my/automl/MyGLMStepsProvider$GLMSteps$1.class(in = 2720) (out= 1267)(deflated 53%)
adding: my/automl/MyDRFStepsProvider$DRFSteps.class(in = 1104) (out= 545)(deflated 50%)
adding: my/automl/MyDRFStepsProvider$DRFSteps$DRFGridStep.class(in = 1308) (out= 644)(deflated 50%)
adding:

In [3]:
# and start the Python client with our plugin
import h2o
h2o.init(extra_classpath=["./dist/h2oautoml_plugin.jar"])

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_241"; Java(TM) SE Runtime Environment (build 1.8.0_241-b07); Java HotSpot(TM) 64-Bit Server VM (build 25.241-b07, mixed mode)
  Starting server from /Users/tomasfryda/sources/h2o-tutorials/tutorials/automl/venv/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/49/kh67vvnj633ftt08t8zfwsvh0000gn/T/tmpaxfdhqe1
  JVM stdout: /var/folders/49/kh67vvnj633ftt08t8zfwsvh0000gn/T/tmpaxfdhqe1/h2o_tomasfryda_started_from_python.out
  JVM stderr: /var/folders/49/kh67vvnj633ftt08t8zfwsvh0000gn/T/tmpaxfdhqe1/h2o_tomasfryda_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Europe/Prague
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,26 days
H2O_cluster_name:,H2O_from_python_tomasfryda_1jugg5
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.547 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


## How to use the custom steps

Those new steps won't be trained by default by `H2O AutoML`, however user can use the `modeling_plan` argument in the `Python` or `R` clients to tell `AutoML` to use them.

Let's first run a simple AutoML job and look at the first modeling steps:

In [4]:
from h2o.automl import H2OAutoML

aml = H2OAutoML(project_name="without_plugin", max_models=20, seed=42)

In [5]:
fr = h2o.import_file("https://h2o-public-test-data.s3.amazonaws.com/smalldata/prostate/prostate.csv")

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [6]:
target = "CAPSULE"
train = fr
train[target] = train[target].asfactor()

In [7]:
aml.train(y=target, training_frame=train)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),4/6
# GBM base models (used / total),0/1
# XGBoost base models (used / total),1/1
# GLM base models (used / total),1/1
# DRF base models (used / total),2/2
# DeepLearning base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5


In [8]:
aml.modeling_steps

[{'name': 'XGBoost', 'steps': [{'id': 'def_2', 'group': 1, 'weight': 10}]},
 {'name': 'GLM', 'steps': [{'id': 'def_1', 'group': 1, 'weight': 10}]},
 {'name': 'GBM', 'steps': [{'id': 'def_5', 'group': 1, 'weight': 10}]},
 {'name': 'XGBoost', 'steps': [{'id': 'def_1', 'group': 2, 'weight': 10}]},
 {'name': 'DRF', 'steps': [{'id': 'def_1', 'group': 2, 'weight': 10}]},
 {'name': 'GBM',
  'steps': [{'id': 'def_2', 'group': 2, 'weight': 10},
   {'id': 'def_3', 'group': 2, 'weight': 10},
   {'id': 'def_4', 'group': 2, 'weight': 10}]},
 {'name': 'XGBoost', 'steps': [{'id': 'def_3', 'group': 3, 'weight': 10}]},
 {'name': 'DRF', 'steps': [{'id': 'XRT', 'group': 3, 'weight': 10}]},
 {'name': 'GBM', 'steps': [{'id': 'def_1', 'group': 3, 'weight': 10}]},
 {'name': 'DeepLearning',
  'steps': [{'id': 'def_1', 'group': 3, 'weight': 10}]},
 {'name': 'XGBoost', 'steps': [{'id': 'grid_1', 'group': 4, 'weight': 90}]},
 {'name': 'GBM', 'steps': [{'id': 'grid_1', 'group': 4, 'weight': 60}]},
 {'name': 'Deep

As we can see, the default run doesn't contain any step defined in our plugin.
To tell AutoML to use our new steps, we will use the `modeling_plan` property.

In [9]:
# we can decide to add our new steps at the beginning: 
# by default, adding just the provider name will add both the default models and the grids.
new_plan = ["MyGLM", "MyDRF"] + aml.modeling_steps

# it is also possible to be more precise when defining the modeling sequence, 
# for example ensuring that default models are all trained before the grids:
another_plan = [
    ('XGBoost', 'defaults'),
    ('GLM', 'defaults'),
    ('DRF', 'defaults'),
    ('GBM', 'defaults'),
    ('DeepLearning', 'defaults'),
    ('MyGLM', 'grids'),
    ('MyDRF', 'grids'),
    ('XGBoost', 'grids'),
    ('GBM', 'grids'),
    ('DeepLearning', 'grids'),
    'StackedEnsemble'
]

# or even go into further details, 
# for example by tweaking the 'weight' property of the `modeling_plan` to produce more models from the `MyGBM` grid, relatively to other grids: 
# this is currently applied only for grids when using `max_runtime_secs` and/or `max_models` constraints. 
yet_another_plan = [
    ('XGBoost', 'defaults'),
    ('GLM', 'defaults'),
    ('DRF', 'defaults'),
    ('GBM', 'defaults'),
    ('DeepLearning', 'defaults'),
    dict(name='MyDRF', steps=[dict(id='grid_1', group=1, weight=100)]),
    dict(name='MyGLM', steps=[dict(id='solvers', group=1, weight=60)]),
    ('GBM', 'grids'),
    ('DeepLearning', 'grids'),
    'StackedEnsemble'
]

In [10]:
new_plan

['MyGLM',
 'MyDRF',
 {'name': 'XGBoost', 'steps': [{'id': 'def_2', 'group': 1, 'weight': 10}]},
 {'name': 'GLM', 'steps': [{'id': 'def_1', 'group': 1, 'weight': 10}]},
 {'name': 'GBM', 'steps': [{'id': 'def_5', 'group': 1, 'weight': 10}]},
 {'name': 'XGBoost', 'steps': [{'id': 'def_1', 'group': 2, 'weight': 10}]},
 {'name': 'DRF', 'steps': [{'id': 'def_1', 'group': 2, 'weight': 10}]},
 {'name': 'GBM',
  'steps': [{'id': 'def_2', 'group': 2, 'weight': 10},
   {'id': 'def_3', 'group': 2, 'weight': 10},
   {'id': 'def_4', 'group': 2, 'weight': 10}]},
 {'name': 'XGBoost', 'steps': [{'id': 'def_3', 'group': 3, 'weight': 10}]},
 {'name': 'DRF', 'steps': [{'id': 'XRT', 'group': 3, 'weight': 10}]},
 {'name': 'GBM', 'steps': [{'id': 'def_1', 'group': 3, 'weight': 10}]},
 {'name': 'DeepLearning',
  'steps': [{'id': 'def_1', 'group': 3, 'weight': 10}]},
 {'name': 'XGBoost', 'steps': [{'id': 'grid_1', 'group': 4, 'weight': 90}]},
 {'name': 'GBM', 'steps': [{'id': 'grid_1', 'group': 4, 'weight': 60

In [11]:
aml_plugin = H2OAutoML(project_name="with_plugin", max_models=20, modeling_plan=new_plan, seed=42)

In [12]:
aml_plugin.train(y=target, training_frame=train)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),5/6
# GBM base models (used / total),0/1
# XGBoost base models (used / total),1/1
# GLM base models (used / total),1/1
# DeepLearning base models (used / total),1/1
# DRF base models (used / total),2/2
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5


In [13]:
aml_plugin_with_weight_and_group = H2OAutoML(project_name="with_plugin_weight_and_group",
                                             max_models=20, modeling_plan=yet_another_plan, seed=42)

In [14]:
aml_plugin_with_weight_and_group.train(y=target, training_frame=train)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),6/6
# GBM base models (used / total),1/1
# XGBoost base models (used / total),1/1
# GLM base models (used / total),1/1
# DRF base models (used / total),2/2
# DeepLearning base models (used / total),1/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5


## Verification

Let's now compare the 2 leaderboards.

The first one contains only models defined by `H2O AutoML`, whereas the second one contains a mix of models defined by both `H2O AutoML` and our plugin.

In [15]:
aml.leaderboard.head(30)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_BestOfFamily_1_AutoML_1_20240409_111346,0.810745,0.528704,0.707779,0.235812,0.417919,0.174657
GLM_1_AutoML_1_20240409_111346,0.808816,0.523744,0.736683,0.273545,0.418759,0.175359
StackedEnsemble_AllModels_1_AutoML_1_20240409_111346,0.803115,0.531748,0.712266,0.244551,0.419681,0.176132
XGBoost_2_AutoML_1_20240409_111346,0.796925,0.546114,0.674287,0.236662,0.425042,0.180661
DRF_1_AutoML_1_20240409_111346,0.793139,0.546391,0.694818,0.270767,0.425796,0.181302
XGBoost_1_AutoML_1_20240409_111346,0.790447,0.546164,0.687742,0.259048,0.426998,0.182327
GBM_4_AutoML_1_20240409_111346,0.78751,0.547597,0.683774,0.258616,0.430242,0.185108
GBM_3_AutoML_1_20240409_111346,0.786934,0.546971,0.691114,0.277879,0.428129,0.183295
XGBoost_grid_1_AutoML_1_20240409_111346_model_3,0.786041,0.56919,0.698224,0.264087,0.435522,0.189679
GBM_1_AutoML_1_20240409_111346,0.784199,0.554973,0.68929,0.255276,0.43086,0.185641


In [16]:
aml_plugin.leaderboard.head(30)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_BestOfFamily_1_AutoML_2_20240409_111554,0.816216,0.517716,0.716022,0.23993,0.413727,0.17117
StackedEnsemble_AllModels_1_AutoML_2_20240409_111554,0.815784,0.516445,0.733739,0.240923,0.413595,0.17106
GLM_1_AutoML_2_20240409_111554,0.808816,0.523744,0.736683,0.273545,0.418759,0.175359
DeepLearning_grid_1_AutoML_2_20240409_111554_model_1,0.805764,0.576212,0.726422,0.257695,0.43227,0.186857
GLM_grid_1_AutoML_2_20240409_111554_model_2,0.805448,0.528366,0.72757,0.271271,0.421004,0.177245
GLM_grid_1_AutoML_2_20240409_111554_model_1,0.805188,0.528585,0.72699,0.264303,0.42119,0.177401
XGBoost_2_AutoML_2_20240409_111554,0.796925,0.546114,0.674287,0.236662,0.425042,0.180661
DRF_1_AutoML_2_20240409_111554,0.793139,0.546391,0.694818,0.270767,0.425796,0.181302
XGBoost_1_AutoML_2_20240409_111554,0.790447,0.546164,0.687742,0.259048,0.426998,0.182327
GBM_4_AutoML_2_20240409_111554,0.78751,0.547597,0.683774,0.258616,0.430242,0.185108


In [17]:
aml_plugin_with_weight_and_group.leaderboard.head(30)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_BestOfFamily_5_AutoML_3_20240409_111636,0.812934,0.519122,0.723725,0.23872,0.414639,0.171926
StackedEnsemble_BestOfFamily_2_AutoML_3_20240409_111636,0.811062,0.525953,0.710498,0.240074,0.416397,0.173386
GLM_1_AutoML_3_20240409_111636,0.808816,0.523744,0.736683,0.273545,0.418759,0.175359
StackedEnsemble_AllModels_1_AutoML_3_20240409_111636,0.807665,0.526937,0.715294,0.234675,0.417351,0.174181
StackedEnsemble_BestOfFamily_1_AutoML_3_20240409_111636,0.807031,0.528696,0.714458,0.23695,0.41878,0.175377
GLM_grid_1_AutoML_3_20240409_111636_model_2,0.805448,0.528366,0.72757,0.271271,0.421004,0.177245
GLM_grid_1_AutoML_3_20240409_111636_model_1,0.805188,0.528585,0.72699,0.264303,0.42119,0.177401
StackedEnsemble_AllModels_5_AutoML_3_20240409_111636,0.804382,0.529274,0.719191,0.247531,0.419218,0.175744
StackedEnsemble_AllModels_2_AutoML_3_20240409_111636,0.80349,0.53249,0.720249,0.251375,0.419972,0.176376
XRT_1_AutoML_3_20240409_111636,0.79717,0.542008,0.700051,0.263597,0.426069,0.181535


In [18]:
h2o.cluster().shutdown()

H2O session _sid_88ad closed.
